### konlpy를 사용한 형태소분리와 Word2Vec를 사용한 벡터변환

In [73]:
# # 한국어 전처리를 위한
!pip install konlpy
# !pip install Twitter
# !pip install tqdm
!pip install gensim
# !pip install --upgrade jupyter ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os # 운영 체제 관련 작업을 수행하기 위한 Python 표준 라이브러리 중 하나 -> 자바 환경 변수를 설정하기 위해 사용
import pandas as pd # 데이터 조작 및 분석을 위한 파이썬 라이브러리 -> 데이터를 데이터프레임으로 로드하고 조작하는 데 사용
import numpy as np # 배열 및 수치 연산을 수행하기 위한 라이브러리 -> 다차원 배열 및 수학적 함수를 다루는 데 사용
from konlpy.tag import Okt # Konlpy 라이브러리에서 제공하는 한국어 형태소 분석기 중 하나 ->  Okt를 초기화하는 데 사용
from gensim.models.word2vec import Word2Vec # Gensim 라이브러리에서 제공하는 Word2Vec 모델을 생성하기 위한 클래스 -> 단어를 고차원 벡터로 표현하여 단어 간 유사성 및 의미 관계를 파악
import re #  Python의 정규 표현식(regular expression) 모듈 -> re.sub() 함수를 사용하여 문자열에서 정규 표현식 패턴에 일치하는 부분을 다른 문자열로 대체
import html # html 엔터티 코드 제거를 위해 사용

In [3]:
# 본인 컴퓨터와 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [4]:
# 데이터를 데이터프레임으로 로드
musical_data = pd.read_csv('../mock_test_csv/mock.csv', encoding='utf-8')# 데이터 파일 경로
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis
0,3885,실연남녀,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/10/13 ~ 2007/12/09,대학로 티오엠 1관,"엄기준,신성록,강지우,양소민,한애리,변우민,성민...",7세이상,NaN,뮤지컬 <실연남녀> 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 ...,어리버리 조폭 형제가 경영하는 “아트 퓨쳐산장”. 이들은 6개월 전 “아트 퓨쳐산장...
1,3635,김종욱 찾기,http://ticketimage.interpark.com/PlayDictionar...,창작,2006/12/12 ~ 2007/04/08,JTN 아트홀 1관,"신성록,전병욱,오나라,안유진,원종환,김재범,김지현...",만 13세이상,90분,“No.1 창작뮤지컬의 신화”뮤지컬 ‘김종욱찾기(연출/김지연)’는 창작뮤지컬계 블루...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디“운명은 항상...
2,4829,첫사랑,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/03/27 ~ 2007/06/17,대학로 티오엠 1관,"조정석,홍광호,해이,전경수,이정섭,김성기,김인수...",만 6세이상,130분,"뮤지컬 <첫사랑>은 어느 바닷가를 배경으로, 이루어지지 못한 첫사랑에 대한 아련한 ...",시간을 조금만 되돌릴 수 있다면…어릴 적부터 한 동네에서 자란 해수와 선이는 오누이...
3,3322,미라클,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/06/01 ~ 2013/02/17,미라클 씨어터 1관,"신현묵,조호균,김고운,원혜지,남궁인,윤도훈,김학규...",만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬!2003년 초연 이래로 16번의 ...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다. 하지만 희...
4,4424,대장금,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/05/26 ~ 2007/06/17,예술의전당 오페라극장,"최보영,안유진,김우형,김소현,이태원,류수화,손광업,한애리,한성식,김나윤,김일우,최윤...",만 7세이상,160분 (인터미션 : 20분),3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...
...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/11/06 ~ 2008/02/03,충무아트센터 소극장 블루,"김우형,윤공주,문성혁,임은영,문지원",7세이상,110분,컨/페/션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑...,그대 가슴속에 두고 와야 할 게 있습니다…그건 바로 사/랑/ 입니다피아노 연주와 신...
96,5270,비보이 퍼포먼스 마리오네트,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/01/12 ~ 2007/03/11,충무아트센터 소극장 블루,NaN,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인!!!타 기존 작품들과의 차별화 성공!2006년 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...
97,5299,마리아 마리아,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/08/23 ~ 2007/08/26,충무아트센터 대극장,"강효성,김보강,이승철",만 8세이상,NaN,2007 뮤지컬 마리아마리아는..뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없...,"ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자, 제사장들은 이에 위..."
98,6509,어린이뮤지컬 장화신은 고양이,http://ticketimage.interpark.com/PlayDictionar...,어린이/가족,2007/07/28 ~ 2007/08/19,국립중앙박물관 극장 용,NaN,만 5세이상,60분,러시아 푸슈킨(Pushkin) 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여...,NaN


In [5]:
# 모든 열에 대해 특수 문자와 HTML 엔터티 코드 모두 제거
for col in musical_data.columns:
    musical_data[col] = musical_data[col].apply(lambda x: re.sub(r'[^\w\s]', '', html.unescape(str(x))))
# 결과 출력
print(musical_data)

   musical_id            title  \
0        3885             실연남녀   
1        3635           김종욱 찾기   
2        4829              첫사랑   
3        3322              미라클   
4        4424              대장금   
..        ...              ...   
95       5075              컨페션   
96       5270   비보이 퍼포먼스 마리오네트   
97       5299          마리아 마리아   
98       6509  어린이뮤지컬 장화신은 고양이   
99       1162            빨간모자2   

                                           poster_url  genre  \
0   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
1   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
2   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
3   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
4   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
..                                                ...    ...   
95  httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
96  httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
97  httpticketi

In [6]:
musical_data.to_csv('../cleaned_data_mock.csv', index=False, encoding='utf-8')

### 텍스트 전처리

In [7]:
# 형태소 분리를 위한 Konlpy 객체 초기화
twitter = Okt()

In [8]:
#텍스트를 전처리 하는 함수
#BOOK_INTRCN_CN: 전처리할 도서 줄거리
def preprocessingText(synopsis):
    #전처리한 결과를 저장 할 리스트
    stems = []
    #text 를 단어로 나누고 품사를 붙여서 리턴
    #stem=True : 단어의 기본형으로 변환  ex) 같습니다,같았습니다 -> 같다.  
    tagged_review = twitter.pos(synopsis,stem=True)

    #tagged에서 하나를 꺼내서 word,pos에 대입
    #단어는 word, 품사는 pos에 대입
    for word,pos in tagged_review:
        #len(word) : 단어의 글자수가 2글자 이상이면
        #품사가 Noun(명사) 또는 Adjective(형용사) 일경우에만 word(단어) 를 stems에 추가
        if pos=="Noun" or pos=='Adjective':
            stems.append(word)

    #리스트인 stems를 문자열로 만들어서 ( "".join(stems) ) 리턴       
    return " ".join(stems)

In [9]:
preprocessingText(musical_data['synopsis'][0])

'조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마비 산장 주인 때문 난감하다 그 은행강도 사건 수배 중인 데 다가 자신 산장 주인 범인 오인 수도 있다 때문 우선 산장 주인 시체 그 대신 산장 운영 새롭다 삶 시작 맹세 그 후 개월 후 손님 교통사고 취소 난감하다 차 여자 손님 지아 남자 손님 연오 손님 좋아하다 그것 잠시 이 두 사람 자살 위해 이 산장 방문 것 알 난감하다 다애 실연 지아 배신 복수 위해 아내 죽음 연오 슬픔 위해 두 사람 추억 장소 아트 퓨쳐 산장 산장 자살 것 목 연오 지아 이 자살 장면 목격 운수 재수 결사 두 사람 만약 이 자살 시체 발견 경찰 수사 것 자신 전적 두렵다 것 자살 실패하다 지아 연오 또 다시 자살 위해 애 운수 재수 지아 연오 자살 위해 온갖 방법 원한 그렇다 가운데 지아 연오 사랑 때문 입장 것 알 그 아픔 이해 연오 역시 지아 아픔 마음 이해 그 연합전선 형성 자신 자살 운수 재수 형제 대항 온갖 죽 방법 원한 마침내 운수 재수 산장 밖 죽 기회 지아 연오 산장 밖 이 것 보고 발 동동 운수 재수 마침내 운수 특단 조치 생각 그 자살 위해 재수 그 비법 과연 이 그 밤 그 원하다 것 손 것 자 자 한바탕 소동 자살 방지 프로젝트 시작'

In [10]:
# 데이터프레임의 'BOOK_INTRCN_CN' 열에 NaN 값이 있을 때 빈 문자열로 대체 및 전처리
musical_data['synopsis_clear'] = musical_data['synopsis'].fillna('').apply(preprocessingText)

In [11]:
# Word2Vec 모델 훈련을 위한 데이터 준비
sentences = musical_data['synopsis'].tolist()

In [12]:
# 데이터프레임의 'synopsis' 열에 NaN 값이 있을 때 빈 문자열로 대체
tokenized_data = musical_data['synopsis'].apply(lambda x: preprocessingText(str(x))).fillna('')

In [13]:
tokenized_data

0     조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...
1     첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...
2     시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...
3     인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...
4     어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...
                            ...                        
95    그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...
96    이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...
97    예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...
98                                                     
99    앤 할머니 시간 뒤 과거 뒤 기차 추억 역 향 추억 역 도착 앤 할머니 마침내 뒤 ...
Name: synopsis, Length: 100, dtype: object

In [14]:
#BOOK_INTRCN_CN_clear의 마지막에 " " 추가
musical_data["synopsis_clear"] = musical_data['synopsis_clear'].astype(str) + " "

In [15]:
#data.split(" ") : 문자열을 공백기준으로 잘라서 리스트로 변환
musical_data["synopsis_clear"].apply(lambda data : data.split(" "))

0     [조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...
1     [첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...
2     [시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...
3     [인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...
4     [어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...
                            ...                        
95    [그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...
96    [이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...
97    [예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...
98                                                 [, ]
99    [앤, 할머니, 시간, 뒤, 과거, 뒤, 기차, 추억, 역, 향, 추억, 역, 도착...
Name: synopsis_clear, Length: 100, dtype: object

In [16]:
#df["BOOK_INTRCN_CN_clear"].apply(lambda data : data.split(" ")) : BOOK_INTRCN_CN_clear 컬럼을 공백을 기준으로 단어별로 분리해서 리턴
musical_data["tokenized_data"] = musical_data["synopsis_clear"].apply(lambda data : data.split(" "))

In [17]:
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,..."
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,..."
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일..."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,..."
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]"


In [18]:
musical_data["tokenized_data"]

0     [조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...
1     [첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...
2     [시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...
3     [인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...
4     [어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...
                            ...                        
95    [그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...
96    [이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...
97    [예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...
98                                                 [, ]
99    [앤, 할머니, 시간, 뒤, 과거, 뒤, 기차, 추억, 역, 향, 추억, 역, 도착...
Name: tokenized_data, Length: 100, dtype: object

### 유사도 측정
- 특정 줄거리와 다른 모든 줄거리 간의 유사도를 측정

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
import random

In [20]:
# 학습용 Word2Vec 모델 훈련(벡터화)
model = Word2Vec(musical_data["tokenized_data"], 
                 vector_size=100, #WordVector의 열의 수
                 window=3, # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=2, # 사용할 단어의 최소 빈도(2회 이하 단어 무시)
                 sg=1) # 0: CBOW, 1: Skip-gram

In [21]:
model.save("word2vec_model2.bin")

In [22]:
#Vector 형태로 변환한 단어 리스트 조회
model.wv.key_to_index

{'그': 0,
 '있다': 1,
 '자신': 2,
 '것': 3,
 '사랑': 4,
 '': 5,
 '그녀': 6,
 '수': 7,
 '위해': 8,
 '없다': 9,
 '사람': 10,
 '이': 11,
 '샌디': 12,
 '예수': 13,
 '말': 14,
 '대니': 15,
 '다시': 16,
 '이야기': 17,
 '집': 18,
 '마리아': 19,
 '존': 20,
 '새롭다': 21,
 '친구': 22,
 '고양이': 23,
 '킴': 24,
 '모습': 25,
 '마음': 26,
 '시작': 27,
 '일': 28,
 '남자': 29,
 '속': 30,
 '서로': 31,
 '렌': 32,
 '날': 33,
 '노래': 34,
 '크리스': 35,
 '그렇다': 36,
 '여자': 37,
 '같다': 38,
 '엄마': 39,
 '둘': 40,
 '춤': 41,
 '하나': 42,
 '때': 43,
 '더': 44,
 '곳': 45,
 '에스메랄다': 46,
 '아이': 47,
 '도시': 48,
 '막': 49,
 '알': 50,
 '한편': 51,
 '생각': 52,
 '삶': 53,
 '마을': 54,
 '안': 55,
 '온': 56,
 '꿈': 57,
 '다른': 58,
 '전': 59,
 '사이': 60,
 '파티': 61,
 '등': 62,
 '두': 63,
 '아름답다': 64,
 '모든': 65,
 '무대': 66,
 '아버지': 67,
 '토니': 68,
 '후': 69,
 '앞': 70,
 '살': 71,
 '순간': 72,
 '클럽': 73,
 '나': 74,
 '모두': 75,
 '자기': 76,
 '공연': 77,
 '소리': 78,
 '어머니': 79,
 '때문': 80,
 '대한': 81,
 '뮤지컬': 82,
 '계속': 83,
 '간다': 84,
 '시간': 85,
 '중': 86,
 '새': 87,
 '학교': 88,
 '음악': 89,
 '사실': 90,
 '생활': 91,
 '소개': 92,
 '듀터': 

In [23]:
#믿음의 인덱스(빈도수 순위 0이 가장 빈도수 높은 단어)
model.wv.key_to_index["할머니"]

508

In [24]:
#믿음과 비슷한 벡터를 가진 단어 조회
model.wv.most_similar("할머니")

[('온', 0.9902034997940063),
 ('그', 0.9899512529373169),
 ('생활', 0.9898141622543335),
 ('이', 0.9897567629814148),
 ('뮤지컬', 0.9897375702857971),
 ('순간', 0.9897342324256897),
 ('없다', 0.9896324276924133),
 ('등', 0.9895016551017761),
 ('못', 0.9893811345100403),
 ('자살', 0.9893527626991272)]

In [25]:
#믿음의 WordVector 조회
model.wv["할머니"]

array([-1.31908972e-02,  3.98894623e-02, -4.81160730e-03, -1.18356850e-02,
        3.02621238e-02, -1.09595142e-01,  3.38012092e-02,  9.18764994e-02,
       -2.33024620e-02, -2.10078079e-02,  1.50913112e-02, -4.37518284e-02,
       -2.13355385e-02,  3.62711325e-02,  1.25970393e-02, -7.61538278e-03,
        2.03465428e-02, -8.63924399e-02, -2.35445728e-03, -7.24246055e-02,
        4.03630771e-02, -4.58750827e-03,  4.67961952e-02, -5.60269803e-02,
       -5.31087723e-03, -2.83358828e-03, -5.56410216e-02, -2.71831919e-02,
       -3.33416089e-02,  1.14731891e-02,  8.92630965e-02,  2.23347303e-02,
        7.44075375e-03, -3.25183496e-02,  2.42258012e-02,  6.68874234e-02,
        1.00516593e-02, -5.86489104e-02, -2.72084493e-02, -7.19399303e-02,
       -3.77784260e-02, -2.43676305e-02, -4.51921746e-02, -1.00266698e-04,
        4.16183546e-02, -3.53771970e-02, -2.10419092e-02,  1.32453015e-05,
        8.21661856e-03,  3.31792235e-02,  7.69073982e-03, -3.47523354e-02,
       -3.07244603e-02, -

In [26]:
#WordVector로 변환한 단어들 word2vec_words 대입
word2vec_words = model.wv.key_to_index.keys()
word2vec_words

dict_keys(['그', '있다', '자신', '것', '사랑', '', '그녀', '수', '위해', '없다', '사람', '이', '샌디', '예수', '말', '대니', '다시', '이야기', '집', '마리아', '존', '새롭다', '친구', '고양이', '킴', '모습', '마음', '시작', '일', '남자', '속', '서로', '렌', '날', '노래', '크리스', '그렇다', '여자', '같다', '엄마', '둘', '춤', '하나', '때', '더', '곳', '에스메랄다', '아이', '도시', '막', '알', '한편', '생각', '삶', '마을', '안', '온', '꿈', '다른', '전', '사이', '파티', '등', '두', '아름답다', '모든', '무대', '아버지', '토니', '후', '앞', '살', '순간', '클럽', '나', '모두', '자기', '공연', '소리', '어머니', '때문', '대한', '뮤지컬', '계속', '간다', '시간', '중', '새', '학교', '음악', '사실', '생활', '소개', '듀터', '노미', '희망', '리조', '행동', '눈', '위', '아니다', '인생', '듀티율', '이제', '궁', '목사', '자살', '못', '밤', '이름', '추억', '동안', '모어', '록시', '또', '자리', '콰', '통해', '손님', '번', '좋아하다', '도착', '가지', '최고', '관계', '점점', '대해', '과연', '모도', '작품', '댄스', '엔지니어', '내', '감정', '주인', '버몬', '장금', '거리', '딸', '향', '벽', '세상', '현실', '어떻다', '일상', '뒤', '역시', '많다', '파의', '젤리', '인하다', '이용', '메리', '소냐', '끝', '분위기', '돈', '캣츠비', '결혼', '마지막', '채', '찰리', '여인', '운명', '페뷔스', '아들', '선', '오해', '지선', 

In [27]:
word2vec_words

dict_keys(['그', '있다', '자신', '것', '사랑', '', '그녀', '수', '위해', '없다', '사람', '이', '샌디', '예수', '말', '대니', '다시', '이야기', '집', '마리아', '존', '새롭다', '친구', '고양이', '킴', '모습', '마음', '시작', '일', '남자', '속', '서로', '렌', '날', '노래', '크리스', '그렇다', '여자', '같다', '엄마', '둘', '춤', '하나', '때', '더', '곳', '에스메랄다', '아이', '도시', '막', '알', '한편', '생각', '삶', '마을', '안', '온', '꿈', '다른', '전', '사이', '파티', '등', '두', '아름답다', '모든', '무대', '아버지', '토니', '후', '앞', '살', '순간', '클럽', '나', '모두', '자기', '공연', '소리', '어머니', '때문', '대한', '뮤지컬', '계속', '간다', '시간', '중', '새', '학교', '음악', '사실', '생활', '소개', '듀터', '노미', '희망', '리조', '행동', '눈', '위', '아니다', '인생', '듀티율', '이제', '궁', '목사', '자살', '못', '밤', '이름', '추억', '동안', '모어', '록시', '또', '자리', '콰', '통해', '손님', '번', '좋아하다', '도착', '가지', '최고', '관계', '점점', '대해', '과연', '모도', '작품', '댄스', '엔지니어', '내', '감정', '주인', '버몬', '장금', '거리', '딸', '향', '벽', '세상', '현실', '어떻다', '일상', '뒤', '역시', '많다', '파의', '젤리', '인하다', '이용', '메리', '소냐', '끝', '분위기', '돈', '캣츠비', '결혼', '마지막', '채', '찰리', '여인', '운명', '페뷔스', '아들', '선', '오해', '지선', 

### 벡터를 문자열로 저장

In [28]:
#행의 수를 row_num에 저장
row_num = len(musical_data)
row_num

100

In [29]:
string_array = []  # 문자열을 저장할 빈 리스트

for index in range(row_num):
    # 뮤지컬 번호 조회
    NUM = musical_data.loc[index,"musical_id"]
    print("NUM=",NUM)
    # 뮤지컬 제목 조회
    TITLE = musical_data.loc[index,"title"]
    print("TITLE=",TITLE)
    # 줄거리 조회
    LINE = musical_data.loc[index,"tokenized_data"]
    print("LINE=",LINE)
    doc2vec = None
    count = 0
    for word in LINE: #줄거리에서 단어 1개씩 리턴
        print("word=",word, end="\t")
        if word in word2vec_words: #단어가 WordVector에 포함된 단어 이면
            count += 1 #개수 1증가
            if doc2vec is None: #doc2vect이 None(초기값 맨처음 실행 첫번째 단어) 이면
                doc2vec = model.wv[word] #단어의 WordVector 값 대입
            else: #첫번째 단어 아님
                doc2vec = doc2vec + model.wv[word] #단어의 WordVector 값 추가

    if  doc2vec is not None:
        # 단어 벡터를 모두 더한 벡터의 값을 단어의 개수로 나눠줌
        doc2vec = doc2vec / count
     # 문자열로 변환해서 string_array 리스트에 추가
    string_array.append(doc2vec.tostring())

# string_array에 저장된 문자열을 데이터프레임에 추가
musical_data["doc2vec_vec"] = string_array

NUM= 3885
TITLE= 실연남녀
LINE= ['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월', '전', '아트', '퓨쳐', '산장', '사채', '빚', '갑자기', '심장마비', '산장', '주인', '때문', '난감하다', '그', '은행강도', '사건', '수배', '중인', '데', '다가', '자신', '산장', '주인', '범인', '오인', '수도', '있다', '때문', '우선', '산장', '주인', '시체', '그', '대신', '산장', '운영', '새롭다', '삶', '시작', '맹세', '그', '후', '개월', '후', '손님', '교통사고', '취소', '난감하다', '차', '여자', '손님', '지아', '남자', '손님', '연오', '손님', '좋아하다', '그것', '잠시', '이', '두', '사람', '자살', '위해', '이', '산장', '방문', '것', '알', '난감하다', '다애', '실연', '지아', '배신', '복수', '위해', '아내', '죽음', '연오', '슬픔', '위해', '두', '사람', '추억', '장소', '아트', '퓨쳐', '산장', '산장', '자살', '것', '목', '연오', '지아', '이', '자살', '장면', '목격', '운수', '재수', '결사', '두', '사람', '만약', '이', '자살', '시체', '발견', '경찰', '수사', '것', '자신', '전적', '두렵다', '것', '자살', '실패하다', '지아', '연오', '또', '다시', '자살', '위해', '애', '운수', '재수', '지아', '연오', '자살', '위해', '온갖', '방법', '원한', '그렇다', '가운데', '지아', '연오', '사랑', '때문', '입장', '것', '알', '그', '아픔', '이해', '연오', '역시', '지아', '아픔', '마음', '이해', '그', '연합전선', '형성', '자신', '자살', 

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_20468\2134378797.py:28: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  string_array.append(doc2vec.tostring())


In [30]:
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\x8a\xa7%\xbd0\x85\x96=T*\xd8;\x87""T\xbc\x8a..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\xb5\xf2\x01\xbd\xcf\xeb\\=\xc7\xc3\xbc;\xba...
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b'O\xf1\xfd\xbc""4c=>\n8;\xa2\x1d$\xbc\xd7P =_\..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'\xa0\x0c\n\xbdt\xe5u=\xb6\x9a\xb1;m\x1f\x1d\...
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...",b'\xe4\xaf\x14\xbd\xfd\n\x85=\xd5\xd5\x98;\xcb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...",b'J\xd65\xbdg\x08\xa0=\x96\x7f\xb1;|=Y\xbc\xa2...
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...","b""u\xcc\x12\xbd\xf2\xfa}=\x96Dg;\xdd\xdd'\xbcx..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...",b'\x8f\xd4\x18\xbd\xa1 \x8e=G7e;\x0c\x171\xbc\...
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]",b'\x84^v\xbd\xf9l\xbd=:\xd1\xbb;0\x17\xd8\xbco...


#### date를 start_date와 end_date로 분리

In [31]:
# 날짜 형식에 맞게 잘라서 "YYYY-MM-DD" 또는 "YYYY-MM" 형식의 문자열로 만듭니다.
musical_data['start_date'] = musical_data['date'].str[:4] + '-' + musical_data['date'].str[4:6] + '-' + musical_data['date'].str[6:8]
musical_data['end_date'] = musical_data['date'].str[:4] + '-' + musical_data['date'].str[4:6]

In [32]:
# datetime 객체로 변환
musical_data['start_date'] = pd.to_datetime(musical_data['start_date'], format="%Y-%m-%d")
musical_data['end_date'] = pd.to_datetime(musical_data['end_date'], format="%Y-%m")

In [33]:
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec,start_date,end_date
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\x8a\xa7%\xbd0\x85\x96=T*\xd8;\x87""T\xbc\x8a...",2007-10-13,2007-10-01
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\xb5\xf2\x01\xbd\xcf\xeb\\=\xc7\xc3\xbc;\xba...,2006-12-12,2006-12-01
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b'O\xf1\xfd\xbc""4c=>\n8;\xa2\x1d$\xbc\xd7P =_\...",2007-03-27,2007-03-01
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'\xa0\x0c\n\xbdt\xe5u=\xb6\x9a\xb1;m\x1f\x1d\...,2007-06-01,2007-06-01
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...",b'\xe4\xaf\x14\xbd\xfd\n\x85=\xd5\xd5\x98;\xcb...,2007-05-26,2007-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...",b'J\xd65\xbdg\x08\xa0=\x96\x7f\xb1;|=Y\xbc\xa2...,2007-11-06,2007-11-01
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...","b""u\xcc\x12\xbd\xf2\xfa}=\x96Dg;\xdd\xdd'\xbcx...",2007-01-12,2007-01-01
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...",b'\x8f\xd4\x18\xbd\xa1 \x8e=G7e;\x0c\x171\xbc\...,2007-08-23,2007-08-01
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]",b'\x84^v\xbd\xf9l\xbd=:\xd1\xbb;0\x17\xd8\xbco...,2007-07-28,2007-07-01


In [35]:
# 'date'와 'location' 사이로 'start_date'와 'end_date' 위치 변경
musical_data = musical_data[['musical_id', 'title', 'poster_url', 'genre', 'date', 'start_date', 'end_date', 'location',
                             'actors', 'age_rating', 'running_time', 'describe', 'synopsis', 'synopsis_clear',
                             'tokenized_data', 'doc2vec_vec']]
musical_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\x8a\xa7%\xbd0\x85\x96=T*\xd8;\x87""T\xbc\x8a..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\xb5\xf2\x01\xbd\xcf\xeb\\=\xc7\xc3\xbc;\xba...
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b'O\xf1\xfd\xbc""4c=>\n8;\xa2\x1d$\xbc\xd7P =_\..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'\xa0\x0c\n\xbdt\xe5u=\xb6\x9a\xb1;m\x1f\x1d\...
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...",b'\xe4\xaf\x14\xbd\xfd\n\x85=\xd5\xd5\x98;\xcb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,2007-11-06,2007-11-01,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...",b'J\xd65\xbdg\x08\xa0=\x96\x7f\xb1;|=Y\xbc\xa2...
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,2007-01-12,2007-01-01,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...","b""u\xcc\x12\xbd\xf2\xfa}=\x96Dg;\xdd\xdd'\xbcx..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,2007-08-23,2007-08-01,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...",b'\x8f\xd4\x18\xbd\xa1 \x8e=G7e;\x0c\x171\xbc\...
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,2007-07-28,2007-07-01,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]",b'\x84^v\xbd\xf9l\xbd=:\xd1\xbb;0\x17\xd8\xbco...


In [36]:
# # musical_data.to_csv('../musical_data_vector.csv', index=False, encoding='utf-8') # 벡터변환 csv저장
# musical_data.to_csv('../mock_test_csv/musical_data_vector2.csv', index=False, encoding='utf-8')

### 추천 시스템

In [37]:
musical_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\x8a\xa7%\xbd0\x85\x96=T*\xd8;\x87""T\xbc\x8a..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\xb5\xf2\x01\xbd\xcf\xeb\\=\xc7\xc3\xbc;\xba...
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b'O\xf1\xfd\xbc""4c=>\n8;\xa2\x1d$\xbc\xd7P =_\..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'\xa0\x0c\n\xbdt\xe5u=\xb6\x9a\xb1;m\x1f\x1d\...
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...",b'\xe4\xaf\x14\xbd\xfd\n\x85=\xd5\xd5\x98;\xcb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,2007-11-06,2007-11-01,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...",b'J\xd65\xbdg\x08\xa0=\x96\x7f\xb1;|=Y\xbc\xa2...
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,2007-01-12,2007-01-01,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...","b""u\xcc\x12\xbd\xf2\xfa}=\x96Dg;\xdd\xdd'\xbcx..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,2007-08-23,2007-08-01,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...",b'\x8f\xd4\x18\xbd\xa1 \x8e=G7e;\x0c\x171\xbc\...
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,2007-07-28,2007-07-01,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]",b'\x84^v\xbd\xf9l\xbd=:\xd1\xbb;0\x17\xd8\xbco...


In [39]:
#wordvector을 문자열로 변환 한 doc2vec_str 컬럼 값을 string_array에 저장
string_array = musical_data.loc[0,"doc2vec_vec"]
string_array

b'\x8a\xa7%\xbd0\x85\x96=T*\xd8;\x87"T\xbc\x8aqP=4\xe4Q\xbe\xb4\x84h=rhH>@\xa4h\xbd+?\xc9\xbc9\xfa\x8c<<e\xa9\xbd\xd8\xed\xbe\xbc\x95\x7f\x98={(\x10=#\xf3\xba\xbc\xa6\x87h=`N \xbe\xc7{3\xbc\x8a\xcb\x02\xbe\xc5f\x92=\x9d5\xb4;g\xe9\xce=L\x1b\xcd\xbd\x12Y/\xbb\xbbz\x89;\x88\x95\xbc\xbd\xe8\xa8\x80\xbd0\xc7L\xbd\xbfa\x9e<\xbc\x8b*>\xdd\xba\xc0<s@\xd6<B\xb4\x90\xbd*z\n=#b\x0e>?\'\xf1;(\x86\xc1\xbd6"e\xbd\x954\x03\xbe\xd4)\x88\xbd\x07\x06y\xbd\xdf\xbc\xb4\xbd\xb2)\xc3\xbb\xfc\'\xb8=\xfcn\x84\xbd]\xcf\r\xbdM\x0cz\xbb\xe2j\xe2;E7\x9d=\x0eM\xe5<\x1c\xa8\x89\xbd\xa9\xbd\x82\xbd\xec\xd5^\xbc\xdb+c\xbc5W\x85=\xaa\xeb\x91=$\xc6\x16<nC\x0b\xbeP\x93\xbf\xbb\x99J\xd1\xbb|\x10\xd4\xbc0:3\xbd\xafa\\=\x9c\xe0 \xbes\x10p=\x12\x96\xde\xbb\x95\xd6i=[\xbf\xcb\xbdv\x9c\x07>]\xdc\x10\xbd\x81\xad\x1c=\x99\x83\x9d=\t\x16\xb9<,)\x81=\xce]\xef<>\x82\x93<\x94\x94\x18\xbd-\x87\x93\xbd[Eo=\xaa\xf8\x00\xbd\x11\xdf0=pW\x90\xbd\xf0\xde\x0b>C\xf6\x07\xbdx\xb6+\xbc&\x7f}\xbc\xa3\xa4\xb0=\xe7\xc2\xdf=X\xcd\x81<\xb2\x8f\xb

In [40]:
# wordvector을 문자열로 변환한  string_array 를 다시 numpy 배열로 변환
np.fromstring(string_array,dtype="float32")

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_20468\1195417262.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np.fromstring(string_array,dtype="float32")


array([-0.04044298,  0.07349622,  0.00659684, -0.01294769,  0.05088953,
       -0.20497209,  0.05676718,  0.19571093, -0.05679727, -0.02456625,
        0.01720916, -0.08271262, -0.02330677,  0.07446209,  0.03519486,
       -0.02282101,  0.05676999, -0.15654898, -0.0109548 , -0.12772956,
        0.07148508,  0.00549956,  0.10103112, -0.10014972, -0.0026756 ,
        0.00419554, -0.09208208, -0.06282216, -0.04999465,  0.01933372,
        0.16654867,  0.0235266 ,  0.02615378, -0.07065631,  0.03380791,
        0.13904624,  0.00735941, -0.09449416, -0.05594083, -0.12813027,
       -0.06648603, -0.06079676, -0.08825087, -0.0059559 ,  0.08992001,
       -0.06466481, -0.03462159, -0.00381543,  0.00690971,  0.07676557,
        0.02799084, -0.06721517, -0.06383831, -0.01360081, -0.01386544,
        0.06510774,  0.07125027,  0.00920251, -0.13599941, -0.00584642,
       -0.00638707, -0.02588677, -0.04375666,  0.0538041 , -0.15710682,
        0.05860944, -0.00679279,  0.05708941, -0.09948608,  0.13

In [42]:
 # wordvector을 문자열로 변환한  string_array 를 다시 numpy 배열로 변환한 결과를 numpy_array에 대입
numpy_array = np.fromstring(string_array,dtype="float32")
numpy_array

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_20468\2740756532.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  numpy_array = np.fromstring(string_array,dtype="float32")


array([-0.04044298,  0.07349622,  0.00659684, -0.01294769,  0.05088953,
       -0.20497209,  0.05676718,  0.19571093, -0.05679727, -0.02456625,
        0.01720916, -0.08271262, -0.02330677,  0.07446209,  0.03519486,
       -0.02282101,  0.05676999, -0.15654898, -0.0109548 , -0.12772956,
        0.07148508,  0.00549956,  0.10103112, -0.10014972, -0.0026756 ,
        0.00419554, -0.09208208, -0.06282216, -0.04999465,  0.01933372,
        0.16654867,  0.0235266 ,  0.02615378, -0.07065631,  0.03380791,
        0.13904624,  0.00735941, -0.09449416, -0.05594083, -0.12813027,
       -0.06648603, -0.06079676, -0.08825087, -0.0059559 ,  0.08992001,
       -0.06466481, -0.03462159, -0.00381543,  0.00690971,  0.07676557,
        0.02799084, -0.06721517, -0.06383831, -0.01360081, -0.01386544,
        0.06510774,  0.07125027,  0.00920251, -0.13599941, -0.00584642,
       -0.00638707, -0.02588677, -0.04375666,  0.0538041 , -0.15710682,
        0.05860944, -0.00679279,  0.05708941, -0.09948608,  0.13

In [43]:
#모든행의 doc2vec_vec 컬럼을 출력
musical_data.loc[: ,"doc2vec_vec"]

0     b'\x8a\xa7%\xbd0\x85\x96=T*\xd8;\x87"T\xbc\x8a...
1     b'\xb5\xf2\x01\xbd\xcf\xeb\\=\xc7\xc3\xbc;\xba...
2     b'O\xf1\xfd\xbc"4c=>\n8;\xa2\x1d$\xbc\xd7P =_\...
3     b'\xa0\x0c\n\xbdt\xe5u=\xb6\x9a\xb1;m\x1f\x1d\...
4     b'\xe4\xaf\x14\xbd\xfd\n\x85=\xd5\xd5\x98;\xcb...
                            ...                        
95    b'J\xd65\xbdg\x08\xa0=\x96\x7f\xb1;|=Y\xbc\xa2...
96    b"u\xcc\x12\xbd\xf2\xfa}=\x96Dg;\xdd\xdd'\xbcx...
97    b'\x8f\xd4\x18\xbd\xa1 \x8e=G7e;\x0c\x171\xbc\...
98    b'\x84^v\xbd\xf9l\xbd=:\xd1\xbb;0\x17\xd8\xbco...
99    b"g\xc7\x1a\xbd\xcb\xa0\x88=\xa6>\xa4;\xb2\x89...
Name: doc2vec_vec, Length: 100, dtype: object

In [44]:
##모든행의 doc2vec_vec 컬럼을 numpy 배열로 변환
musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_20468\1483775256.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )


0     [-0.04044298, 0.07349622, 0.006596843, -0.0129...
1     [-0.031725604, 0.053935822, 0.005760643, -0.01...
2     [-0.030998854, 0.05546964, 0.0028082277, -0.01...
3     [-0.033703446, 0.060033277, 0.0054200543, -0.0...
4     [-0.036300555, 0.064962365, 0.0046641626, -0.0...
                            ...                        
95    [-0.044393815, 0.078141026, 0.0054168208, -0.0...
96    [-0.035839517, 0.0620069, 0.0035288683, -0.010...
97    [-0.037312087, 0.06939817, 0.0034975575, -0.01...
98    [-0.06014873, 0.092493005, 0.005731729, -0.026...
99    [-0.03778782, 0.06671294, 0.005012351, -0.0137...
Name: doc2vec_vec, Length: 100, dtype: object

In [45]:
##모든행의 doc2vec_vec 컬럼을 numpy 배열로 변환해서 doc2vec_numpy 컬럼에 대입
musical_data.loc[: ,"doc2vec_numpy"] = musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_20468\2972792042.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  musical_data.loc[: ,"doc2vec_numpy"] = musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )
C:\Users\dsmdm\AppData\Local\Temp\ipykernel_20468\2972792042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musical_data.loc[: ,"doc2vec_numpy"] = musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )


In [46]:
musical_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec,doc2vec_numpy
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\x8a\xa7%\xbd0\x85\x96=T*\xd8;\x87""T\xbc\x8a...","[-0.04044298, 0.07349622, 0.006596843, -0.0129..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\xb5\xf2\x01\xbd\xcf\xeb\\=\xc7\xc3\xbc;\xba...,"[-0.031725604, 0.053935822, 0.005760643, -0.01..."
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b'O\xf1\xfd\xbc""4c=>\n8;\xa2\x1d$\xbc\xd7P =_\...","[-0.030998854, 0.05546964, 0.0028082277, -0.01..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'\xa0\x0c\n\xbdt\xe5u=\xb6\x9a\xb1;m\x1f\x1d\...,"[-0.033703446, 0.060033277, 0.0054200543, -0.0..."
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...",b'\xe4\xaf\x14\xbd\xfd\n\x85=\xd5\xd5\x98;\xcb...,"[-0.036300555, 0.064962365, 0.0046641626, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,2007-11-06,2007-11-01,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...",b'J\xd65\xbdg\x08\xa0=\x96\x7f\xb1;|=Y\xbc\xa2...,"[-0.044393815, 0.078141026, 0.0054168208, -0.0..."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,2007-01-12,2007-01-01,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...","b""u\xcc\x12\xbd\xf2\xfa}=\x96Dg;\xdd\xdd'\xbcx...","[-0.035839517, 0.0620069, 0.0035288683, -0.010..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,2007-08-23,2007-08-01,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...",b'\x8f\xd4\x18\xbd\xa1 \x8e=G7e;\x0c\x171\xbc\...,"[-0.037312087, 0.06939817, 0.0034975575, -0.01..."
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,2007-07-28,

In [48]:
# doc2vec_numpy 배열의 값을 리스트로 변환
musical_data["doc2vec_numpy"].tolist()

[array([-0.04044298,  0.07349622,  0.00659684, -0.01294769,  0.05088953,
        -0.20497209,  0.05676718,  0.19571093, -0.05679727, -0.02456625,
         0.01720916, -0.08271262, -0.02330677,  0.07446209,  0.03519486,
        -0.02282101,  0.05676999, -0.15654898, -0.0109548 , -0.12772956,
         0.07148508,  0.00549956,  0.10103112, -0.10014972, -0.0026756 ,
         0.00419554, -0.09208208, -0.06282216, -0.04999465,  0.01933372,
         0.16654867,  0.0235266 ,  0.02615378, -0.07065631,  0.03380791,
         0.13904624,  0.00735941, -0.09449416, -0.05594083, -0.12813027,
        -0.06648603, -0.06079676, -0.08825087, -0.0059559 ,  0.08992001,
        -0.06466481, -0.03462159, -0.00381543,  0.00690971,  0.07676557,
         0.02799084, -0.06721517, -0.06383831, -0.01360081, -0.01386544,
         0.06510774,  0.07125027,  0.00920251, -0.13599941, -0.00584642,
        -0.00638707, -0.02588677, -0.04375666,  0.0538041 , -0.15710682,
         0.05860944, -0.00679279,  0.05708941, -0.0

In [49]:
#doc2vec_numpy 배열의 값을 리스트로 변환 한 값을 numpy 배열로 변환
np.array(musical_data["doc2vec_numpy"].tolist() )

array([[-0.04044298,  0.07349622,  0.00659684, ..., -0.14661428,
        -0.00590961, -0.0285509 ],
       [-0.0317256 ,  0.05393582,  0.00576064, ..., -0.10695284,
        -0.0048116 , -0.02291895],
       [-0.03099885,  0.05546964,  0.00280823, ..., -0.11243653,
        -0.00582263, -0.02090558],
       ...,
       [-0.03731209,  0.06939817,  0.00349756, ..., -0.13473599,
        -0.00537456, -0.02439888],
       [-0.06014873,  0.09249301,  0.00573173, ..., -0.18761282,
        -0.00513318, -0.04199219],
       [-0.03778782,  0.06671294,  0.00501235, ..., -0.12690923,
        -0.0045814 , -0.02543792]], dtype=float32)

In [51]:
from sklearn.preprocessing import StandardScaler
#데이터의 각 열의 평균을 뺀 다음 표준편차로 나눠서 평균을 0로 표준편차를 1로 변환하는 StandardScaler 객체 생성
scaler = StandardScaler()

In [52]:
#각 열의 평균 표준편차 계산
scaler.fit(np.array(musical_data["doc2vec_numpy"].tolist() ))

StandardScaler()

In [53]:
#각 열의 평균
scaler.mean_

array([-0.0423115 ,  0.07369563,  0.00527565, -0.01437463,  0.04984962,
       -0.20522519,  0.05590642,  0.19642014, -0.0573501 , -0.02499068,
        0.01534211, -0.08449204, -0.02324577,  0.0743885 ,  0.0334208 ,
       -0.02367306,  0.05514327, -0.15632912, -0.0106003 , -0.12788045,
        0.07127843,  0.0047364 ,  0.10131618, -0.0993655 , -0.00237236,
        0.00450043, -0.09123344, -0.06442666, -0.05086377,  0.01961077,
        0.16736841,  0.02216775,  0.02758517, -0.07250975,  0.03296384,
        0.13847478,  0.00842176, -0.09518983, -0.05490189, -0.1273168 ,
       -0.06530603, -0.0637184 , -0.09042989, -0.00711161,  0.09036893,
       -0.06444483, -0.03412692, -0.00285607,  0.0085169 ,  0.07552807,
        0.02914315, -0.06791486, -0.06290134, -0.01244665, -0.01535033,
        0.06449444,  0.07067174,  0.00634972, -0.1368628 , -0.00497478,
       -0.00624906, -0.02593017, -0.04168236,  0.053934  , -0.15703694,
        0.06092337, -0.00672017,  0.057193  , -0.09964876,  0.13

In [54]:
#각 열의 분산
scaler.var_

array([7.31096283e-05, 1.54045812e-04, 1.71761529e-06, 1.98262681e-05,
       6.10313375e-05, 1.16799507e-03, 9.83813773e-05, 1.10288503e-03,
       1.06564218e-04, 2.86008198e-05, 5.05535617e-06, 2.13612426e-04,
       2.71026980e-05, 1.39106555e-04, 2.71975561e-05, 3.21955350e-05,
       7.37248016e-05, 7.17920588e-04, 5.17604205e-06, 4.69804002e-04,
       1.18807715e-04, 1.11029654e-06, 3.20654331e-04, 2.46175108e-04,
       6.12376360e-06, 1.76645026e-06, 2.19947230e-04, 1.39620891e-04,
       7.39958850e-05, 1.99069260e-05, 7.97831773e-04, 1.45387288e-05,
       1.78832728e-05, 1.77552095e-04, 2.91368009e-05, 5.84956874e-04,
       6.95796850e-06, 2.39272781e-04, 8.11208690e-05, 4.64780285e-04,
       1.02893437e-04, 1.33658178e-04, 2.57507202e-04, 2.60820124e-06,
       2.27032755e-04, 1.26769430e-04, 3.25947744e-05, 5.94338324e-06,
       3.20990415e-06, 1.53256363e-04, 3.43982070e-05, 1.51674198e-04,
       9.51208417e-05, 6.87158217e-06, 2.16061172e-05, 1.20341780e-04,
      

In [56]:
#데이터의 각 열의 평균을 뺀 다음 표준편차로 나눠서 평균을 0로 표준편차를 1로 변환 한 후 리스트로 변환해서 doc2vec_numpy_scale 컬럼에 대입
musical_data["doc2vec_numpy_scale"] = scaler.transform(np.array(musical_data["doc2vec_numpy"].tolist() )).tolist()

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_20468\1926048544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musical_data["doc2vec_numpy_scale"] = scaler.transform(np.array(musical_data["doc2vec_numpy"].tolist() )).tolist()


In [57]:
musical_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec,doc2vec_numpy,doc2vec_numpy_scale
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\x8a\xa7%\xbd0\x85\x96=T*\xd8;\x87""T\xbc\x8a...","[-0.04044298, 0.07349622, 0.006596843, -0.0129...","[0.21852977573871613, -0.016066240146756172, 1..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\xb5\xf2\x01\xbd\xcf\xeb\\=\xc7\xc3\xbc;\xba...,"[-0.031725604, 0.053935822, 0.005760643, -0.01...","[1.2380563020706177, -1.5920538902282715, 0.37..."
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b'O\xf1\xfd\xbc""4c=>\n8;\xa2\x1d$\xbc\xd7P =_\...","[-0.030998854, 0.05546964, 0.0028082277, -0.01...","[1.323052167892456, -1.4684737920761108, -1.88..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'\xa0\x0c\n\xbdt\xe5u=\xb6\x9a\xb1;m\x1f\x1d\...,"[-0.033703446, 0.060033277, 0.0054200543, -0.0...","[1.0067410469055176, -1.1007800102233887, 0.11..."
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...",b'\xe4\xaf\x14\xbd\xfd\n\x85=\xd5\xd5\x98;\xcb...,"[-0.036300555, 0.064962365, 0.0046641626, -0.0...","[0.7030003666877747, -0.7036418318748474, -0.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,2007-11-06,2007-11-01,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...",b'J\xd65\xbdg\x08\xa0=\x96\x7f\xb1;|=Y\xbc\xa2...,"[-0.044393815, 0.078141026, 0.0054168208, -0.0...","[-0.24353358149528503, 0.35816705226898193, 0...."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,2007-01-12,2007-01-01,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...","b""u\xcc\x12\xbd\xf2\xfa}=\x96Dg;\xdd\xdd'\xbcx...","[-0.035839517, 0.0620069, 0.0035288683, -0.010...","[0.756920337677002, -0.9417647123336792, -1.33..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,2007-08-23,2007-08-01,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리

In [58]:
# doc2vec_numpy_scale 컬럼 조회
musical_data["doc2vec_numpy_scale"] 

0     [0.21852977573871613, -0.016066240146756172, 1...
1     [1.2380563020706177, -1.5920538902282715, 0.37...
2     [1.323052167892456, -1.4684737920761108, -1.88...
3     [1.0067410469055176, -1.1007800102233887, 0.11...
4     [0.7030003666877747, -0.7036418318748474, -0.4...
                            ...                        
95    [-0.24353358149528503, 0.35816705226898193, 0....
96    [0.756920337677002, -0.9417647123336792, -1.33...
97    [0.5846983194351196, -0.34624743461608887, -1....
98    [-2.0861239433288574, 1.5145103931427002, 0.34...
99    [0.5290595889091492, -0.5625975728034973, -0.2...
Name: doc2vec_numpy_scale, Length: 100, dtype: object

In [59]:
from sklearn.metrics.pairwise import euclidean_distances
# doc2vec_numpy_scale 컬럼의 유클리드 거리를 계산
sim_score = euclidean_distances(musical_data["doc2vec_numpy_scale"].tolist(),musical_data["doc2vec_numpy_scale"].tolist()) 

In [60]:
sim_score

array([[0.00000000e+00, 1.40543204e+01, 1.40463112e+01, ...,
        6.55447944e+00, 1.74863365e+01, 7.65159980e+00],
       [1.40543204e+01, 5.33120150e-07, 4.07640472e+00, ...,
        1.04103252e+01, 2.81479272e+01, 7.51140344e+00],
       [1.40463112e+01, 4.07640472e+00, 4.12953092e-07, ...,
        1.02303029e+01, 2.80681514e+01, 7.37898009e+00],
       ...,
       [6.55447944e+00, 1.04103252e+01, 1.02303029e+01, ...,
        2.06476546e-07, 2.10306361e+01, 5.65615152e+00],
       [1.74863365e+01, 2.81479272e+01, 2.80681514e+01, ...,
        2.10306361e+01, 4.76837158e-07, 2.21765898e+01],
       [7.65159980e+00, 7.51140344e+00, 7.37898009e+00, ...,
        5.65615152e+00, 2.21765898e+01, 3.15398134e-07]])

In [61]:
# sim_score (synopsys_vector_numpy_scale 컬럼의 유클리드 거리) DataFrame 으로 변환
sim_df = pd.DataFrame(data=sim_score)

In [62]:
sim_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,1.405432e+01,1.404631e+01,1.143953e+01,8.312022e+00,4.547696,5.393616,3.885476,4.878147,14.672811,...,11.725550,16.551787,1.748634e+01,5.863351,5.991505,3.780287e+00,1.023888e+01,6.554479e+00,1.748634e+01,7.651600e+00
1,14.054320,5.331201e-07,4.076405e+00,4.809184e+00,7.131718e+00,11.729216,9.892247,16.197800,11.042159,2.987898,...,4.757784,4.793315,2.814793e+01,8.860872,9.335071,1.588231e+01,5.427366e+00,1.041033e+01,2.814793e+01,7.511403e+00
2,14.046311,4.076405e+00,4.129531e-07,4.430765e+00,6.827788e+00,11.497315,9.836143,15.903534,11.002346,3.110116,...,3.780262,5.075912,2.806815e+01,8.942176,8.967375,1.573238e+01,4.981709e+00,1.023030e+01,2.806815e+01,7.378980e+00
3,11.439532,4.809184e+00,4.430765e+00,1.685874e-07,4.999155e+00,9.297713,7.707727,13.534332,8.340778,4.742839,...,3.827828,6.664693,2.573002e+01,6.534966,6.900043,1.320468e+01,3.390397e+00,8.190940e+00,2.573002e+01,5.113651e+00
4,8.312022,7.131718e+00,6.827788e+00,4.999155e+00,2.920019e-07,6.211434,4.990456,10.179320,5.713422,7.680096,...,5.375264,9.669521,2.271702e+01,4.065827,4.167841,9.927704e+00,4.350366e+00,6.161470e+00,2.271702e+01,3.614013e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3.780287,1.588231e+01,1.573238e+01,1.320468e+01,9.927704e+00,5.232344,6.798807,2.641642,5.956342,16.540982,...,13.298163,18.516866,1.587160e+01,7.652997,7.769876,5.960464e-08,1.181894e+01,7.678853e+00,1.587160e+01,9.410915e+00
96,10.238883,5.427366e+00,4.981709e+00,3.390397e+00,4.350366e+00,7.922236,6.374294,12.028528,7.295519,5.340992,...,3.388665,7.594484,2.443600e+01,5.071098,5.525086,1.181894e+01,2.920019e-07,6.371014e+00,2.443600e+01,3.789125e+00
97,6.554479,1.041033e+01,1.023030e+01,8.190940e+00,6.161470e+00,4.491994,4.448775,7.891517,5.525657,11.040175,...,8.223968,12.858368,2.103064e+01,4.291589,4.876043,7.678853e+00,6.371014e+00,2.064765e-07,2.103064e+01,5.656152e+00
98,17.486336,2.814793e+01,2.806815e+01,2.573002e+01,2.271702e+01,19.209726,20.412801,15.881322,19.294656,28.419494,...,26.044877,30.534319,4.768372e-07,20.930728,21.001877,1.587160e+01,2.443600e+01,2.103064e+01,4.768372e-07,2.217659e+01


In [65]:
#sin_df의 인덱스에 영화 제목 대입
sim_df.index = musical_data["title"]
sim_df.columns = musical_data["title"]
sim_df

title,실연남녀,김종욱 찾기,첫사랑,미라클,대장금,렌트,댄싱 섀도우,블루다이아몬드,테너를 빌려줘,스트리트 가이즈,...,펌프보이즈,어린이뮤지컬 퍼니팜,어린이뮤지컬 앤서니 브라운의 돼지책,화성에서 꿈꾸다,우리동네,컨페션,비보이 퍼포먼스 마리오네트,마리아 마리아,어린이뮤지컬 장화신은 고양이,빨간모자2
title,,,,,,,,,,,,,,,,,,,,,
실연남녀,0.000000,1.405432e+01,1.404631e+01,1.143953e+01,8.312022e+00,4.547696,5.393616,3.885476,4.878147,14.672811,...,11.725550,16.551787,1.748634e+01,5.863351,5.991505,3.780287e+00,1.023888e+01,6.554479e+00,1.748634e+01,7.651600e+00
김종욱 찾기,14.054320,5.331201e-07,4.076405e+00,4.809184e+00,7.131718e+00,11.729216,9.892247,16.197800,11.042159,2.987898,...,4.757784,4.793315,2.814793e+01,8.860872,9.335071,1.588231e+01,5.427366e+00,1.041033e+01,2.814793e+01,7.511403e+00
첫사랑,14.046311,4.076405e+00,4.129531e-07,4.430765e+00,6.827788e+00,11.497315,9.836143,15.903534,11.002346,3.110116,...,3.780262,5.075912,2.806815e+01,8.942176,8.967375,1.573238e+01,4.981709e+00,1.023030e+01,2.806815e+01,7.378980e+00
미라클,11.439532,4.809184e+00,4.430765e+00,1.685874e-07,4.999155e+00,9.297713,7.707727,13.534332,8.340778,4.742839,...,3.827828,6.664693,2.573002e+01,6.534966,6.900043,1.320468e+01,3.390397e+00,8.190940e+00,2.573002e+01,5.113651e+00
대장금,8.312022,7.131718e+00,6.827788e+00,4.999155e+00,2.920019e-07,6.211434,4.990456,10.179320,5.713422,7.680096,...,5.375264,9.669521,2.271702e+01,4.065827,4.167841,9.927704e+00,4.350366e+00,6.161470e+00,2.271702e+01,3.614013e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
컨페션,3.780287,1.588231e+01,1.573238e+01,1.320468e+01,9.927704e+00,5.232344,6.798807,2.641642,5.956342,16.540982,...,13.298163,18.516866,1.587160e+01,7.652997,7.769876,5.960464e-08,1.181894e+01,7.678853e+00,1.587160e+01,9.410915e+00
비보이 퍼포먼스 마리오네트,10.238883,5.427366e+00,4.981709e+00,3.390397e+00,4.350366e+00,7.922236,6.374294,12.028528,7.295519,5.340992,...,3.388665,7.594484,2.443600e+01,5.071098,5.525086,1.181894e+01,2.920019e-07,6.371014e+00,2.443600e+01,3.789125e+00
마리아 마리아,6.554479,1.041033e+01,1.023030e+01,8.190940e+00,6.161470e+00,4.491994,4.448775,7.891517,5.525657,11.040175,...,8.223968,12.858368,2.103064e+01,4.291589,4.876043,7.678853e+00,6.371014e+00,2.064765e-07,2.103064e+01,5.656152e+00


In [67]:
# 특정 뮤지컬 작품과의 거리가 가까준 순으로 5개 조회
sim_df["빨간모자2"].sort_values()[0:5]

title
빨간모자2       3.153981e-07
화성에서 꿈꾸다    3.298726e+00
스위니토드       3.343462e+00
피크닉         3.420826e+00
지하철 1호선     3.455470e+00
Name: 빨간모자2, dtype: float64